In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from goto import with_goto
import time
import pandas as pd 

In [3]:
def find_jobs(driver,jobs,num_jobs,verbose):

            time.sleep(15)


        
            try:
                driver.find_element(By.XPATH,".//*[contains(@class,'icl-CloseButton')]").click()
                time.sleep(10)
            except:
                pass
        
            job_buttons = driver.find_elements(By.XPATH,".//*[@class='jobsearch-ResultsList css-0']//li//div[contains(@class,'cardOutline')]")  
            
            for job_button in job_buttons:  

                print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
                if len(jobs) >= num_jobs:
                    print("Completed")
                    break
                
                
                try:
                    driver.find_element(By.XPATH,".//*[contains(@class,'icl-CloseButton')]").click()
                    time.sleep(10)
                except:
                    pass

                job_button.click()  

                

                time.sleep(2)
                collected_successfully = False

                
                
                while not collected_successfully:
                    try:
                        company_name = driver.find_element(By.XPATH,'.//div[@data-company-name="true"]//a').text
                        location = driver.find_element(By.XPATH,'.//div[contains(@class,"vjs-highlight")]//.//div[@class="companyLocation"]').text
                        job_title = driver.find_element(By.XPATH,'.//*[contains(@class,"jobsearch-JobInfoHeader-title")]//span').text
                        job_title = job_title.split("\n", 1)[0]
                        
                        try:
                            job_description = driver.find_element(By.XPATH,'.//*[contains(@class,"jobsearch-JobComponent-description")]').text  
                        except NoSuchElementException:
                            job_description = -1
                        collected_successfully = True
                    except:
                        time.sleep(5)

                try:
                    salary_estimate = driver.find_element(By.XPATH, '//div[@id="salaryInfoAndJobType"]//span').text
                except NoSuchElementException:
                    salary_estimate = -1 
                
                try:
                    benefits_and_perks=driver.find_elements(By.XPATH,'.//div[contains(@class,"css-1oqmop4")]//div//span')
                    benefits_and_perks_ls=[]
                    for bnf in benefits_and_perks:
                        bnf=bnf.text
                        try:
                            bnf=bnf.replace(",","")
                        except:
                            pass
                        benefits_and_perks_ls.append(bnf)
                    
                except:
                    benefits_and_perks_ls="N/A"


                try:
                    job_types=driver.find_elements(By.XPATH,'.//div[contains(@class,"css-rr5fiy")]//div')
                    job_types_ls=[]
                    for jt in job_types:
                        jt=jt.text
                        try:
                            jt=jt.replace(",","")
                        except:
                            pass
                        job_types_ls.append(jt)
                    job_types_ls.remove("Job Type")
                except:
                    job_types_ls="N/A"

                try:
                    rating = driver.find_element(By.XPATH,'.//div[contains(@class,"vjs-highlight")]//.//span[@class="ratingNumber"]').text
                except NoSuchElementException:
                    rating = -1 

                
                if verbose:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))
                    print("benefits and perks: {}".format(benefits_and_perks_ls))
                    print("Job Type: {}".format(job_types_ls))
                    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")


                jobs.append({"Job Title" : job_title,
                "Salary Estimate" : salary_estimate,
                "Job Description" : job_description,
                "Rating" : rating,
                "Company Name" : company_name,
                "Location" : location,
                "Benefits and perks": benefits_and_perks_ls,
                "Job Type": job_types_ls,
                })

In [4]:
def get_jobs(keyword,location="", num_jobs=5, verbose=False):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))# 
    
    driver.set_window_size(1120, 1000)
   
    url = 'https://in.indeed.com/?from=gnav-jobsearch--indeedmobile'
    driver.get(url)


    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="yosegi-InlineWhatWhere-searchBox"]')))
    search_box = driver.find_element(By.XPATH,'//input[@id="text-input-what"]')
    search_box.send_keys(keyword)
    
    if location=="":
        search_button = driver.find_element(By.XPATH,'//*[class="yosegi-InlineWhatWhere-primaryButton"]')
        search_button.click()
    else:
        location_box = driver.find_element(By.XPATH,'//*[@id="text-input-where"]')
        location_box.click()
        location_box.clear()
        location_box.send_keys(location)
        search_button = driver.find_element(By.XPATH,'//*[@class="yosegi-InlineWhatWhere-primaryButton"]')
        search_button.click()



    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="jobsearch-ResultsList css-0"]')))
    jobs = []

    while len(jobs) < num_jobs:
        
        find_jobs(driver,jobs,num_jobs,verbose)
        
        try:
            driver.find_element(By.XPATH,'.//*[@data-testid="pagination-page-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)

In [5]:
from alive_progress import alive_bar
import time